# Scraping Data from spbuir.pl

Getting the data and importing libraries

In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

url = "https://spbuir.pl/lista-brokerow"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Data frame 
cols = ['name', 'company', 'city', 'email', 'phone', 'cell_phone']
data_frame = pd.DataFrame(columns=cols)

# Table
table = soup.select('#lista_brokerow')[0]

## Getting data (name, company, city)

In [2]:
# Rows
column_data = table.find_all('tr')

# Looping all rows except the first one
for row in column_data[1:]:
  row_data = row.find_all('td')
  individual_row_data = [data.text.strip() for data in row_data]

  # Cleaning first column '1. \xa0\xa0Katarzyna Abramczuk'
  if individual_row_data:
        individual_row_data[0] = individual_row_data[0][5:].strip()

  data_frame.loc[len(data_frame)] = {'name': individual_row_data[0],
                                     'company': individual_row_data[1],
                                     'city': individual_row_data[2]}
  

## Getting data (email, phone, cell_phone)

In [3]:
# Initialize an index for the DataFrame
index = 0

# Looping all rows except the first one
for row in column_data[1:]:
    # Getting the link for more details
    details_link = row.find('a')['href']

    # Now, you can use requests to get information from the details page
    details_response = requests.get(details_link)
    details_soup = BeautifulSoup(details_response.text, 'html.parser')

    # Acceding all p tags
    p_tags = details_soup.find_all('p')

    # Access the fourth p tag
    fourth_p_tag = p_tags[3]

    # Extract text from the p tag
    p_text = fourth_p_tag.get_text(separator="\n")

    # Split the text into lines
    lines = p_text.split("\n")

    # Extract email, phone, and cell phone from the lines
    email = lines[2]
    phone = lines[3]
    cell_phone = lines[-1]

    # Add to DataFrame
    data_frame.loc[index, ['email', 'phone', 'cell_phone']] = [email, phone, cell_phone]
    
    # Increment the index
    index += 1


C:\Users\gomez\AppData\Local\Temp\ipykernel_10488\3072868.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'broker@abramczuk.pl' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_frame.loc[index, ['email', 'phone', 'cell_phone']] = [email, phone, cell_phone]
C:\Users\gomez\AppData\Local\Temp\ipykernel_10488\3072868.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Telefon: 83 344 22 44' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_frame.loc[index, ['email', 'phone', 'cell_phone']] = [email, phone, cell_phone]
C:\Users\gomez\AppData\Local\Temp\ipykernel_10488\3072868.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Telefon komórkowy: 504102142' has dtype incompatible wi

## Cleaning data

In [4]:
data_frame['phone'] = data_frame['phone'].str.replace('Telefon: ', '')
data_frame['cell_phone'] = data_frame['cell_phone'].str.replace('Telefon komórkowy: ', '')

## Exporting data

In [5]:
file_name = 'output.csv'
data_frame.to_csv(file_name, index=False, sep=';')